# Utility and convenience methods

## Imports

In [8]:
import logging
import rdflib
import pyshacl
import rdflib.tools.rdf2dot
import graphviz
import io
import json

logging.basicConfig(level=logging.INFO)

## Load from text and files

In [9]:
RDF_FORMATS = {
    "jsonld": "json-ld",
    "json":"json-ld",
    "json-ld":"json-ld",
    "ttl":"turtle",
    "turtle":"turtle"
}

def guessFileFormat(fname):
    parts = fname.rsplit(".", 1)
    gformat = None
    try:
        ext = parts[1].lower()
        gformat = RDF_FORMATS.get(ext, None)
    except e as Exception:
        logging.warning(e)
    return gformat

def guessSourceFormat(source):
    src = source.strip()
    try:
        test = json.loads(source)
        logging.debug("Guessed type as JSON-LD")
        return "json-ld"
    except json.JSONDecodeError:
        pass
    if (src[-1] == "."):
        logging.debug("Guessed type as Turtle")
        return "turtle"
    return None

def loadGraph(fname, gformat, publicID):
    if gformat is None:
        gformat = guessFileFormat(fname)
    g = rdflib.ConjunctiveGraph()
    return g.parse(fname, format=gformat, publicID=publicID)

def loadShaclGraph(fname, format=None, publicID="https://science-on-schema.org/examples/shacl/"):
    return loadGraph(fname, format, publicID)

def loadDataGraph(fname, format=None, publicID="https://science-on-schema.org/examples/data/"):
    return loadGraph(fname, format, publicID)

def parseGraph(source, gformat, publicID):
    if gformat is None:
        gformat = guessSourceFormat(source)
    g = rdflib.ConjunctiveGraph()
    return g.parse(data=source, format=gformat, publicID=publicID)

def parseShaclGraph(source, format=None, publicID="https://science-on-schema.org/examples/shacl/"):
    return parseGraph(source, format, publicID)

def parseDataGraph(source, format=None, publicID="https://science-on-schema.org/examples/data/"):
    return parseGraph(source, format, publicID)

def visualizeGraph(g):
    fp = io.StringIO()
    rdflib.tools.rdf2dot.rdf2dot(g, fp)
    return graphviz.Source( fp.getvalue() )    

# Validation

In [10]:
def validate(data_graph, shacl_graph, ont_graph=None, advanced=True, inference="rdfs"):
    return pyshacl.validate(data_graph, shacl_graph=shacl_graph, ont_graph=ont_graph, advanced=advanced, inference=inference)

def validateAndPrint(data_graph, shacl_graph, ont_graph=None, advanced=True, inference="rdfs"):
    def printMsg(msg, v):
      print(f"{msg}\n{'-'*len(msg)}\n{v}\n")
    
    # Apply the SHACL rules
    conforms, report_graph, report_text = validate(data_graph, shacl_graph=shacl_graph, ont_graph=ont_graph, advanced=advanced, inference=inference)
    printMsg("Conforms", conforms)
    printMsg("Report Text", report_text)
    printMsg("Report Graph", report_graph.serialize(format="turtle", indent=2).decode())
    return conforms, report_graph, report_text
    